# Convolutional Networks Architectures

In [33]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary

from func import train

In [34]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Dataset

In [35]:
BATCH_SIZE = 256
transforms = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])
train_dataset = tv.datasets.MNIST('./datas', train=True, transform=transforms, download=True)
test_dataset = tv.datasets.MNIST('./datas', train=False, transform=transforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

## AlexNet

In [42]:
net = nn.Sequential(
    nn.Conv2d(1, 96, kernel_size=11, stride=4),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Conv2d(96, 256, kernel_size=5, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Conv2d(256, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Flatten(),
    nn.Linear(6400, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 10)
)

In [43]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          11,712
              ReLU-2           [-1, 96, 54, 54]               0
         MaxPool2d-3           [-1, 96, 26, 26]               0
            Conv2d-4          [-1, 256, 26, 26]         614,656
              ReLU-5          [-1, 256, 26, 26]               0
         MaxPool2d-6          [-1, 256, 12, 12]               0
            Conv2d-7          [-1, 384, 12, 12]         885,120
              ReLU-8          [-1, 384, 12, 12]               0
            Conv2d-9          [-1, 384, 12, 12]       1,327,488
             ReLU-10          [-1, 384, 12, 12]               0
           Conv2d-11          [-1, 256, 12, 12]         884,992
             ReLU-12          [-1, 256, 12, 12]               0
        MaxPool2d-13            [-1, 256, 5, 5]               0
          Flatten-14                 [-

In [41]:
lr, num_epochs = 0.001, 1
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, optimizer, num_epochs, device)

Step 0, time since epoch: 0.182. Train acc: 0.121. Train Loss: 2.303


Step 10, time since epoch: 2.136. Train acc: 0.111. Train Loss: 2.323
Step 20, time since epoch: 3.897. Train acc: 0.193. Train Loss: 2.187
Step 30, time since epoch: 5.672. Train acc: 0.333. Train Loss: 1.844
Step 40, time since epoch: 7.463. Train acc: 0.436. Train Loss: 1.581
Step 50, time since epoch: 9.250. Train acc: 0.517. Train Loss: 1.365
Step 60, time since epoch: 11.064. Train acc: 0.577. Train Loss: 1.198
Step 70, time since epoch: 12.911. Train acc: 0.622. Train Loss: 1.075
Step 80, time since epoch: 14.731. Train acc: 0.662. Train Loss: 0.968
Step 90, time since epoch: 16.516. Train acc: 0.693. Train Loss: 0.880
Step 100, time since epoch: 18.290. Train acc: 0.718. Train Loss: 0.808
Step 110, time since epoch: 20.063. Train acc: 0.738. Train Loss: 0.752
Step 120, time since epoch: 21.835. Train acc: 0.756. Train Loss: 0.702
Step 130, time since epoch: 23.606. Train acc: 0.772. Train Loss: 0.660
Step 140, time since epoch: 25.400. Train acc: 0.785. Train Loss: 0.621
Step 1

## VGG